 # <span style="color:green">__Universidad EAN:</span> Facultad de Ingenieria - Maestría en Ciencias de Datos__
 ## **<span style="color:orange">Procesamiento de Lenguaje Natural</span>**

* __Geraldine Giseth Patiño Pardo__
* __Juan Sebastian Lozano Forero__
* __Natalia Rico Torres__

# __1. Importación y Carga de Librerias__

In [1]:
import re ## Exprexiones regulares
import nltk ## Procesamiento de lenguaje natural
import unicodedata
from nltk.corpus import stopwords ## Palabras vacias
from nltk.stem import SnowballStemmer ## Stemming
from nltk.tokenize import word_tokenize ## Tokenizacion
from nltk.tokenize import RegexpTokenizer ## Tokenizacion
from sklearn.feature_extraction.text import CountVectorizer ## Vectorizador
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy ## Procesamiento de lenguaje natural

############## Descarga de recursos de nltk ################
nltk.download('punkt') ## Tokenizador
nltk.download('punk_tab')
nltk.download('stopwords') ## Palabras vacias
nltk.download('snowball_data') ## Stemming
nltk.download('wordnet') ## Lematizacion

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# __2. Carga de base de datos__

In [2]:
## Se carga la base de datos
import pandas as pd

Booking_df=pd.read_csv('https://raw.githubusercontent.com/Izainea/nlp_ean/refs/heads/main/Datos/Datos%20Crudos/reviews_booking.csv')
Booking_df

<ipython-input-2-ee02bb7a9e33>:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Booking_df=pd.read_csv('https://raw.githubusercontent.com/Izainea/nlp_ean/refs/heads/main/Datos/Datos%20Crudos/reviews_booking.csv')


,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10,"Ubicación, comodidades y limpieza",NaN,Sí
2,2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...
438208,438208,Estados Unidos,Apartamento Suite,1 noche,septiembre de 2024,En familia,22 de octubre de 2024,Fantástico,"9,0",NaN,NaN,No
438209,438209,Colombia,Apartamento de 1 dormitorio,1 noche,octubre de 2024,Persona que viaja sola,9 de octubre de 2024,Excepcional,10,NaN,NaN,No
438210,438210,México,Apartamento de 1 dormitorio,4 noches,agosto de 2024,Persona que viaja sola,6 de septiembre de 2024,Pésimo,"1,0",No me pareció nada,No obstante que dos días antes de mi llegada a...,Sí
438211,438211,Colombia,Apartamento de 1 dormitorio,2 noches,julio de 2024,En pareja,22 de julio de 2024,Excepcional,10,NaN,NaN,No


# __3. Se generan Dataframes con reseñas de Colombia tanto positivas como negativas__

Se extrajeron los comentarios positivos y negativos basados en la calificacion dada por el usuario.
*  Positivo calificacion mayor o igual a 9
*  Negativo calificacion menor o igual a 7

Adicionalmente se se clasifico la calidad de los comentarios segun su longitud siendo esta mayor a 60 caracteres.

In [3]:
# Filtrar y eliminar los registros donde 'reseña' sea "No" y 'País' sea "Colombia"
BookingCol = Booking_df[(Booking_df['reseña'] == 'Sí') & (Booking_df['País'] == 'Colombia')]

In [4]:
BookingCol['Calificación']=BookingCol['Calificación'].str.replace(',','.').astype(float)

<ipython-input-4-cc6632f0cd66>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BookingCol['Calificación']=BookingCol['Calificación'].str.replace(',','.').astype(float)


In [5]:
DF_inicial=BookingCol[BookingCol['Calificación']<=7]
DF_inicial=DF_inicial['Cosas Negativas']
DF_inicial=DF_inicial[DF_inicial.fillna('').apply(len)>60].sample(7500)
DF_inicial

,Cosas Negativas
430389,El tema de las toallas y el kit de aseo. Éramo...
269595,Desayuno faltó variedad y algo de comida típic...
376695,proceso para asignacion del parqueadero muy co...
354322,TODO EL HOSTAL APESTA A CIGARRILLO! LLAMESE HA...
345569,Definitivamente la comida del buffet es muy ma...
...,...
71871,No me gusto para nada la atención en la recepc...
244269,El sifón del lavamanos y el sifón de la ducha ...
337176,"No respetaron la reserva, era una habitación p..."
241213,toda la noche habia habian ruidos de aparatos ...


In [6]:
DF_Final=pd.DataFrame()

DF_Final['Comentarios']=DF_inicial
DF_Final['Clas']='Negativos'
DF_Final

,Comentarios,Clas
430389,El tema de las toallas y el kit de aseo. Éramo...,Negativos
269595,Desayuno faltó variedad y algo de comida típic...,Negativos
376695,proceso para asignacion del parqueadero muy co...,Negativos
354322,TODO EL HOSTAL APESTA A CIGARRILLO! LLAMESE HA...,Negativos
345569,Definitivamente la comida del buffet es muy ma...,Negativos
...,...,...
71871,No me gusto para nada la atención en la recepc...,Negativos
244269,El sifón del lavamanos y el sifón de la ducha ...,Negativos
337176,"No respetaron la reserva, era una habitación p...",Negativos
241213,toda la noche habia habian ruidos de aparatos ...,Negativos


In [7]:
DF_inicial2=BookingCol[BookingCol['Calificación']>=9]

DF_inicial2=DF_inicial2['Cosas Positivas']
DF_inicial2=DF_inicial2[DF_inicial2.fillna('').apply(len)>60].sample(7500)
DF_inicial2

,Cosas Positivas
260318,Las instalaciones impecables y el personal muy...
118967,"Instalaciones muy bien cuidadas y limpias, ate..."
331800,"Todo muy limpio y ordenado, la habitación exce..."
97585,"Maravillosa atención del personal, trato atent..."
288163,Excelente relación calidad/precio. Excepcional...
...,...
174060,Cordialidad de las señoritas Miriam y Nicoll d...
117987,"El desayuno bastante completo y muy delicioso,..."
23769,El hotel en el medio de la zona G está suprema...
152096,La ubicación es excelente \n En toda la zona d...


In [8]:
DF_Final_2=pd.DataFrame()

DF_Final_2['Comentarios']=DF_inicial2
DF_Final_2['Clas']='Positivos'
DF_Final_2

,Comentarios,Clas
260318,Las instalaciones impecables y el personal muy...,Positivos
118967,"Instalaciones muy bien cuidadas y limpias, ate...",Positivos
331800,"Todo muy limpio y ordenado, la habitación exce...",Positivos
97585,"Maravillosa atención del personal, trato atent...",Positivos
288163,Excelente relación calidad/precio. Excepcional...,Positivos
...,...,...
174060,Cordialidad de las señoritas Miriam y Nicoll d...,Positivos
117987,"El desayuno bastante completo y muy delicioso,...",Positivos
23769,El hotel en el medio de la zona G está suprema...,Positivos
152096,La ubicación es excelente \n En toda la zona d...,Positivos


In [9]:
# Combinar los dataframes apilando filas
DF_BookingCol = pd.concat([DF_Final, DF_Final_2], ignore_index=True)

# Verificar el resultado
print(DF_BookingCol.shape)

(15000, 2)


Tras el proceso de seleccion de los comentarios positivos y negativos, adicional a filtrar de acuerdo a la lonigtud del comentario se tiene una data set de 15000 registros.

# __4. A continuación se ejecutará cada los siguientes pasos dentro del procesamiento de texto:__
* __4.1. Normalización de texto__
* __4.2. Tokenización__
* __4.3. Eliminación de stopwords__
* __4.4. Stemming y lematización__
* __4.5. Bag of words__

## 4.1. Normalización de texto

In [10]:
# Función para limpiar caracteres y eliminar tildes
def limpiar_caracteres_no_espanol_sin_tildes(texto):
    # Eliminar caracteres especiales y palabras en alfabetos no latinos
    texto = re.sub(r'[^\d\w\sñÑ]', '', texto)  # Solo deja palabras y espacios
    texto = re.sub(r'[^\u0000-\u007F\u00C0-\u017F\sñÑ]', '', texto)  # Filtrar caracteres no latinos

    # Normalizar y eliminar tildes
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = re.sub(r'[\u0300-\u036f]', '', texto).strip()  # Eliminar marcas de acento (diacríticos)

    return texto

# Aplicar al DataFrame
DF_BookingCol['Texto_Limpio'] = DF_BookingCol['Comentarios'].apply(limpiar_caracteres_no_espanol_sin_tildes)

# Ver el resultado
print(DF_BookingCol[['Comentarios', 'Texto_Limpio']].head())


                                         Comentarios  \
0  El tema de las toallas y el kit de aseo. Éramo...   
1  Desayuno faltó variedad y algo de comida típic...   
2  proceso para asignacion del parqueadero muy co...   
3  TODO EL HOSTAL APESTA A CIGARRILLO! LLAMESE HA...   
4  Definitivamente la comida del buffet es muy ma...   

                                        Texto_Limpio  
0  El tema de las toallas y el kit de aseo Eramos...  
1  Desayuno falto variedad y algo de comida tipic...  
2  proceso para asignacion del parqueadero muy co...  
3  TODO EL HOSTAL APESTA A CIGARRILLO LLAMESE HAB...  
4  Definitivamente la comida del buffet es muy ma...  


In [11]:
# Función actualizada para eliminar números y normalizar texto
def normalizar_texto_completamente(texto):
    if isinstance(texto, str):  # Asegurarse de procesar solo texto
        # Eliminar números
        texto = re.sub(r'\d', '', texto)
        # Eliminar caracteres especiales y normalizar tildes
        texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
        # Eliminar espacios adicionales
        texto = re.sub(r'\s+', ' ', texto).strip()
        return texto
    return texto  # Si no es texto, devolver sin cambios

# Aplicar la función de limpieza
DF_BookingCol['Texto_Limpio'] = DF_BookingCol['Texto_Limpio'].apply(normalizar_texto_completamente)

## 4.2. Tokenización

In [13]:
# Asegúrate de descargar el recurso necesario
nltk.download('punkt')
nltk.download('punkt_tab')
# Aplicar la tokenización por palabras a cada fila de la columna 'Comentarios'
DF_BookingCol['Tokens'] = DF_BookingCol['Texto_Limpio'].apply(word_tokenize)

# Ver el resultado
print(DF_BookingCol[['Comentarios', 'Texto_Limpio', 'Tokens']])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


                                             Comentarios  \
0      El tema de las toallas y el kit de aseo. Éramo...   
1      Desayuno faltó variedad y algo de comida típic...   
2      proceso para asignacion del parqueadero muy co...   
3      TODO EL HOSTAL APESTA A CIGARRILLO! LLAMESE HA...   
4      Definitivamente la comida del buffet es muy ma...   
...                                                  ...   
14995  Cordialidad de las señoritas Miriam y Nicoll d...   
14996  El desayuno bastante completo y muy delicioso,...   
14997  El hotel en el medio de la zona G está suprema...   
14998  La ubicación es excelente \n En toda la zona d...   
14999  El personal del hotel fue supremamente atento!...   

                                            Texto_Limpio  \
0      El tema de las toallas y el kit de aseo Eramos...   
1      Desayuno falto variedad y algo de comida tipic...   
2      proceso para asignacion del parqueadero muy co...   
3      TODO EL HOSTAL APESTA A CIGARRIL

## 4.3. Eliminación de stopwords

In [14]:
# Stopwords en español e inglés como conjuntos
stopwords_esp = set(stopwords.words('spanish'))  # Convertir a conjunto
stopwords_eng = set(stopwords.words('english'))  # Convertir a conjunto

# Combinar ambas listas de stopwords
stopwords_combined = stopwords_esp.union(stopwords_eng)  # Unir ambos conjuntos

# Verificar la combinación
print(f"Total stopwords combinadas: {len(stopwords_combined)}")

# Eliminar stopwords en ambos idiomas
DF_BookingCol['Sin_Stopwords_Combined'] = DF_BookingCol['Tokens'].apply(
    lambda tokens: [word for word in tokens if word.lower() not in stopwords_combined]
)

Total stopwords combinadas: 485


In [20]:
import spacy
!python -m spacy download es_core_news_sm
nlp = spacy.load('es_core_news_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 72.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [21]:
# Cargar el modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

In [22]:
DF_BookingCol['Lematizado_Combined'] = DF_BookingCol['Sin_Stopwords_Combined'].apply(
    lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))]
)

# Ver el resultado
print(DF_BookingCol[['Lematizado_Combined','Sin_Stopwords_Combined']])

                                     Lematizado_Combined  \
0      [tema, toalla, kit, aseo, Eramos, dos, persona...   
1      [desayuno, falto, variedad, comida, tipico, pa...   
2      [proceso, asignacion, parqueadero, complicado,...   
3      [hostal, APESTA, CIGARRILLO, LLAMESE, HABITACI...   
4      [definitivamente, comida, buffet, mala, buen, ...   
...                                                  ...   
14995  [cordialidad, senorita, Miriam, Nicoll, recepc...   
14996  [desayunar, bastante, completo, delicioso, ate...   
14997  [hotel, medio, zona, G, supremamente, bien, ub...   
14998  [ubicacion, excelente, todo, zona, rós, Bogota...   
14999  [personal, hotel, supremamente, atento, querid...   

                                  Sin_Stopwords_Combined  
0      [tema, toallas, kit, aseo, Eramos, dos, person...  
1      [Desayuno, falto, variedad, comida, tipica, pa...  
2      [proceso, asignacion, parqueadero, complicado,...  
3      [HOSTAL, APESTA, CIGARRILLO, LLAMESE

## 4.4. Stemming y Lemantización

 __Para este caso de estudio se determinó que la manera más adecuada para reducir las palabras a su forma base es la lemmatización ya que con este método se considera su significado y contexto gramatical, lo cual es una ventaja para el análisis de los comentarios realizados por las personas en Booking__

In [23]:
# Cargar el modelo de spaCy en español
nlp = spacy.load('es_core_news_sm')

In [24]:
DF_BookingCol['Lematizado_Combined'] = DF_BookingCol['Sin_Stopwords_Combined'].apply(
    lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))]
)

# Ver el resultado
print(DF_BookingCol[['Sin_Stopwords_Combined','Lematizado_Combined']])

                                  Sin_Stopwords_Combined  \
0      [tema, toallas, kit, aseo, Eramos, dos, person...   
1      [Desayuno, falto, variedad, comida, tipica, pa...   
2      [proceso, asignacion, parqueadero, complicado,...   
3      [HOSTAL, APESTA, CIGARRILLO, LLAMESE, HABITACI...   
4      [Definitivamente, comida, buffet, mala, buen, ...   
...                                                  ...   
14995  [Cordialidad, senoritas, Miriam, Nicoll, recep...   
14996  [desayuno, bastante, completo, delicioso, aten...   
14997  [hotel, medio, zona, G, supremamente, bien, ub...   
14998  [ubicacion, excelente, toda, zona, rosa, Bogot...   
14999  [personal, hotel, supremamente, atento, querid...   

                                     Lematizado_Combined  
0      [tema, toalla, kit, aseo, Eramos, dos, persona...  
1      [desayuno, falto, variedad, comida, tipico, pa...  
2      [proceso, asignacion, parqueadero, complicado,...  
3      [hostal, APESTA, CIGARRILLO, LLAMESE

# __5. Se aplica modelo de Bag of Words__

## 5.1. CountVectorizer

In [25]:
# Convertir las listas de lemas a textos unificados
DF_BookingCol['Lematizado_unido'] = DF_BookingCol['Lematizado_Combined'].apply(lambda tokens: " ".join(tokens))

# Inicializar el vectorizador
vectorizer = CountVectorizer()

# Aplicar BoW a la columna 'Texto_Lematizado'
bow_matrix = vectorizer.fit_transform(DF_BookingCol['Lematizado_unido'])

# Convertir la matriz de BoW a un DataFrame para una visualización más fácil
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Mostrar el resultado
print(bow_df)


       aaael  aangeven  aangevir  aanwezig  aao  ab  abajo  abandonado  \
0          0         0         0         0    0   0      0           0   
1          0         0         0         0    0   0      0           0   
2          0         0         0         0    0   0      0           0   
3          0         0         0         0    0   0      0           0   
4          0         0         0         0    0   0      0           0   
...      ...       ...       ...       ...  ...  ..    ...         ...   
14995      0         0         0         0    0   0      0           0   
14996      0         0         0         0    0   0      0           0   
14997      0         0         0         0    0   0      0           0   
14998      0         0         0         0    0   0      0           0   
14999      0         0         0         0    0   0      0           0   

       abandonar  abandono  ...  zullys  zum  zumo  zuruck  zusatzlicher  \
0              0         0  ...    

## 5.2. TF-IDF (Term Frequency-Inverse Document Frequency):

In [29]:
# Unir los lemas en textos completos para cada fila
DF_BookingCol['Texto_Lematizado'] = DF_BookingCol['Lematizado_Combined'].apply(lambda tokens: " ".join(tokens))

# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Ajustar y transformar los textos
tfidf_matrix = tfidf_vectorizer.fit_transform(DF_BookingCol['Texto_Lematizado'])

# Convertir la matriz a un DataFrame para visualizarla
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Mostrar los primeros resultados
print(tfidf_df.head())

   aaael  aangeven  aangevir  aanwezig  aao   ab  abajo  abandonado  \
0    0.0       0.0       0.0       0.0  0.0  0.0    0.0         0.0   
1    0.0       0.0       0.0       0.0  0.0  0.0    0.0         0.0   
2    0.0       0.0       0.0       0.0  0.0  0.0    0.0         0.0   
3    0.0       0.0       0.0       0.0  0.0  0.0    0.0         0.0   
4    0.0       0.0       0.0       0.0  0.0  0.0    0.0         0.0   

   abandonar  abandono  ...  zullys  zum  zumo  zuruck  zusatzlicher  \
0        0.0       0.0  ...     0.0  0.0   0.0     0.0           0.0   
1        0.0       0.0  ...     0.0  0.0   0.0     0.0           0.0   
2        0.0       0.0  ...     0.0  0.0   0.0     0.0           0.0   
3        0.0       0.0  ...     0.0  0.0   0.0     0.0           0.0   
4        0.0       0.0  ...     0.0  0.0   0.0     0.0           0.0   

   zwischenubernachtung  zón        él   ón   ús  
0                   0.0  0.0  0.000000  0.0  0.0  
1                   0.0  0.0  0.000000

## 5.3. Identificación de frecuencia de las palabras:

In [38]:
# Sumar las ocurrencias de cada palabra
palabra_frecuencias_B = bow_df.sum().sort_values(ascending=False)
print(palabra_frecuencias_B.head(9))  # Top 10 palabras más frecuentes


# Sumar las ocurrencias de cada palabra usando TF IDF
palabra_frecuencias_T = tfidf_df.sum().sort_values(ascending=False)
print(palabra_frecuencias_T.head(9))  # Top 10 palabras más frecuentes

#Tabla de comparacion de BOW y TDF-IDF las primeras 10 palabras en orden
resultado_df = pd.DataFrame({
    'Frecuencia BOW': palabra_frecuencias_B.head(9),
    'Frecuencia TDF-IDF': palabra_frecuencias_T.head(9)
})

resultado_df

habitacion    4874
hotel         3889
personal      3260
desayuno      3189
excelente     2688
atencion      2609
ubicacion     2549
buen          2285
servicio      2082
dtype: int64
habitacion    545.202431
personal      518.949010
excelente     512.546753
desayuno      480.920341
ubicacion     475.654233
atencion      463.779544
hotel         439.742446
buen          421.933314
servicio      344.057684
dtype: float64


,Frecuencia BOW,Frecuencia TDF-IDF
atencion,2609,463.779544
buen,2285,421.933314
desayuno,3189,480.920341
excelente,2688,512.546753
habitacion,4874,545.202431
hotel,3889,439.742446
personal,3260,518.949010
servicio,2082,344.057684
ubicacion,2549,475.654233


# __6. Conclusiones__
###A partir del proceso realizado y al comparar los resultados se puede ver que las palabras de mayor aparicion en los textos analizados son:###

* habitacion    
* hotel         
* personal      
* desayuno      
* excelente     
* atencion      
* ubicacion     
* buen          
* servicio

####Sin embargo cuando se aplica el modelo TF-IDF  la palabra hotel a pesar de tener el 2do puesto en total de apariciones realmente no tiene tanto peso dentro de los comentarios.